In [1]:
# Some steps required until these Python modules are properly installed...
import sys
sys.path.append('../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
import veneer
import gbr
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
#To do: alter the path to the relevant model output directory
#path = 'L:\Source Projects\Fitzroy\RC2015\Model Outputs\Fitzroy RC2015'
path = 'D:/Fitzroy/Results/Fitzroy Rebuild 2014/'
path = path.replace('"\"','"/"')
gbr.init(path)
gbr.available()

['Baseline_3818_OrigParams',
 'FROM_ROB',
 'FROM_ROB.zip',
 'TEST_REARRANGE_PARALLEL',
 'TEST_REPORTING_SHORT_RUN',
 'TEST_REPORTING_SPEED',
 'TIMING_WITHOUT_PARAC_WITHOUT_PARAN_WITHOUT_PRERUN_WITHOUT_PARAF',
 'TIMING_WITHOUT_PARAC_WITHOUT_PARAN_WITH_PRERUN_WITHOUT_PARAF',
 'TIMING_WITHOUT_PARAC_WITH_PARAN_WITHOUT_PRERUN_WITHOUT_PARAF',
 'TIMING_WITHOUT_PARAC_WITH_PARAN_WITH_PRERUN_WITHOUT_PARAF',
 'TIMING_WITH_PARAC_WITHOUT_PARAN_WITHOUT_PRERUN_WITHOUT_PARAF',
 'TIMING_WITH_PARAC_WITHOUT_PARAN_WITHOUT_PRERUN_WITH_PARAF',
 'TIMING_WITH_PARAC_WITHOUT_PARAN_WITH_PRERUN_WITHOUT_PARAF',
 'TIMING_WITH_PARAC_WITHOUT_PARAN_WITH_PRERUN_WITH_PARAF',
 'TIMING_WITH_PARAC_WITH_PARAN_WITHOUT_PRERUN',
 'TIMING_WITH_PARAC_WITH_PARAN_WITHOUT_PRERUN_WITHOUT_PARAF',
 'TIMING_WITH_PARAC_WITH_PARAN_WITHOUT_PRERUN_WITH_PARAF',
 'TIMING_WITH_PARAC_WITH_PARAN_WITH_PRERUN',
 'TIMING_WITH_PARAC_WITH_PARAN_WITH_PRERUN_WITHOUT_PARAF',
 'TIMING_WITH_PARAC_WITH_PARAN_WITH_PRERUN_WITH_PARAF']

In [3]:
#To do: select the model results from the list above and copy into the line below
results = gbr.Results('Baseline_3818_OrigParams')

In [4]:
raw = results.get('RawResults')

In [5]:
#Remove budget elements not required (the last 3 refer to flow which is removed)
filteredRaw = raw[~raw.BudgetElement.isin(['Link Intial Load', 'Link In Flow', 'Link Yield', 'Node Yield', 'Evaporation', 'Infiltration', 'Rainfall'])]

In [6]:
grouped_rawResults = filteredRaw.reset_index().groupby(['Process', 'BudgetElement', 'Constituent']).sum()

In [7]:
massBalanceKg = pd.pivot_table(grouped_rawResults.reset_index(),
                               index = ['Process','BudgetElement'],
                               columns = 'Constituent',
                               values = 'Total_Load_in_Kg')
#massBalanceKg

In [8]:
years = results.runDetails.yearsOfRecording
years

4.076659822039699

In [9]:
#massBalanceKgAnnual = massBalanceKg/results.runDetails.yearsOfRecording
#Use 28 years instead - this is the number that has been used in manual calcs in the past
massBalanceKgAnnual = massBalanceKg/4.08

In [10]:
#Convert units
massBalanceAnnual = massBalanceKgAnnual.copy()
conversions = {'Sediment - Fine':1e-6,'Flow':1e-6,'N_DIN':1e-3,'N_DON':1e-3,
'N_Particulate':1e-3,'P_DOP':1e-3,'P_FRP':1e-3,'P_Particulate':1e-3}
for col in massBalanceAnnual.columns:
    if col in conversions:
        massBalanceAnnual[col] *= conversions[col]
#massBalanceAnnual

In [11]:
#Reorder colums to the expected arrangement...
# A bit tricky...
col_order = ['Sediment - Fine','Sediment - Coarse','P_Particulate','P_FRP','P_DOP','N_Particulate','N_DIN','N_DON']
other_cols = list(set(massBalanceAnnual.columns)-set(col_order))
massBalanceAnnual = massBalanceAnnual[col_order + other_cols]

massBalanceAnnual

Constituent                               Sediment - Fine  Sediment - Coarse  \
Process  BudgetElement                                                         
Loss     Extraction                            563.051320       0.000000e+00   
         Flood Plain Deposition                  0.000000                NaN   
         Node Loss                            1331.856676       0.000000e+00   
         Reservoir Decay                              NaN                NaN   
         Reservoir Deposition                 1361.746308       0.000000e+00   
         Stream Decay                                 NaN                NaN   
         Stream Deposition                       0.000000       9.431344e+08   
Other    Leached                                      NaN                NaN   
Residual Residual Link Storage                  15.393046       5.368681e-04   
         Residual Node Storage                 296.663904       0.000000e+00   
Supply   Channel Remobilisation                  0.000000       0.000000e+00   
         Diffuse Dissolved                            NaN                NaN   
         Gully                                3693.380018       0.000000e+00   
         Hillslope no source distinction              NaN                NaN   
         Hillslope sub-surface soil              0.000000       0.000000e+00   
         Hillslope surface soil               1640.586204       1.370189e+08   
         Link Initial Load                       0.000000       0.000000e+00   
         Node Initial Load                       0.000000       0.000000e+00   
         Point Source                                 NaN                NaN   
         Seepage                                      NaN                NaN   
         Streambank                           1267.237651       8.062049e+08   
         Undefined                                    NaN                NaN   

Constituent                               P_Particulate        P_FRP  \
Process  BudgetElement                                                 
Loss     Extraction                         1199.083494    56.127939   
         Flood Plain Deposition                0.000000          NaN   
         Node Loss                          1923.651565   199.258786   
         Reservoir Decay                            NaN     0.000000   
         Reservoir Deposition               1954.967535          NaN   
         Stream Decay                               NaN     0.000000   
         Stream Deposition                     0.000000          NaN   
Other    Leached                                    NaN          NaN   
Residual Residual Link Storage                18.751432     4.434158   
         Residual Node Storage               145.712604    62.570108   
Supply   Channel Remobilisation                0.000000          NaN   
         Diffuse Dissolved                          NaN  1420.531577   
         Gully                              1104.361263          NaN   
         Hillslope no source distinction    6098.053482          NaN   
         Hillslope sub-surface soil                 NaN          NaN   
         Hillslope surface soil                     NaN          NaN   
         Link Initial Load                     0.000000     0.000000   
         Node Initial Load                     0.000000     0.000000   
         Point Source                               NaN    32.944679   
         Seepage                                    NaN          NaN   
         Streambank                          535.066403          NaN   
         Undefined                           983.889665          NaN   

Constituent                                    P_DOP  N_Particulate  \
Process  BudgetElement                                                
Loss     Extraction                        11.111050    1747.557719   
         Flood Plain Deposition                  NaN       0.000000   
         Node Loss                         38.002298    2806.49193

In [12]:
# Calculate the category totals...

# Easier to work with a transposed table
massBalanceAnnual_transposed=massBalanceAnnual.transpose()
elements = ['Supply','Loss','Residual','Other']
for element in elements:
    massBalanceAnnual_transposed['Total',element] = massBalanceAnnual.ix[element].fillna(0).sum()

massBalanceAnnual_transposed['Total','Export'] = 2*massBalanceAnnual_transposed['Total','Supply']-massBalanceAnnual_transposed['Total'].transpose().sum()

# Then transpose back...
massBalanceAnnual_transposed2 = massBalanceAnnual_transposed.transpose()
massBalanceAnnual_transposed2

Constituent                               Sediment - Fine  Sediment - Coarse  \
Process  BudgetElement                                                         
Loss     Extraction                            563.051320       0.000000e+00   
         Flood Plain Deposition                  0.000000                NaN   
         Node Loss                            1331.856676       0.000000e+00   
         Reservoir Decay                              NaN                NaN   
         Reservoir Deposition                 1361.746308       0.000000e+00   
         Stream Decay                                 NaN                NaN   
         Stream Deposition                       0.000000       9.431344e+08   
Other    Leached                                      NaN                NaN   
Residual Residual Link Storage                  15.393046       5.368681e-04   
         Residual Node Storage                 296.663904       0.000000e+00   
Supply   Channel Remobilisation                  0.000000       0.000000e+00   
         Diffuse Dissolved                            NaN                NaN   
         Gully                                3693.380018       0.000000e+00   
         Hillslope no source distinction              NaN                NaN   
         Hillslope sub-surface soil              0.000000       0.000000e+00   
         Hillslope surface soil               1640.586204       1.370189e+08   
         Link Initial Load                       0.000000       0.000000e+00   
         Node Initial Load                       0.000000       0.000000e+00   
         Point Source                                 NaN                NaN   
         Seepage                                      NaN                NaN   
         Streambank                           1267.237651       8.062049e+08   
         Undefined                                    NaN                NaN   
Total    Supply                               6601.203874       9.432239e+08   
         Loss                                 3256.654304       9.431344e+08   
         Residual                              312.056950       5.368681e-04   
         Other                                   0.000000       0.000000e+00   
         Export                               3032.492620       8.950619e+04   

Constituent                               P_Particulate        P_FRP  \
Process  BudgetElement                                                 
Loss     Extraction                         1199.083494    56.127939   
         Flood Plain Deposition                0.000000          NaN   
         Node Loss                          1923.651565   199.258786   
         Reservoir Decay                            NaN     0.000000   
         Reservoir Deposition               1954.967535          NaN   
         Stream Decay                               NaN     0.000000   
         Stream Deposition                     0.000000          NaN   
Other    Leached                                    NaN          NaN   
Residual Residual Link Storage                18.751432     4.434158   
         Residual Node Storage               145.712604    62.570108   
Supply   Channel Remobilisation                0.000000          NaN   
         Diffuse Dissolved                          NaN  1420.531577   
         Gully                              1104.361263          NaN   
         Hillslope no source distinction    6098.053482          NaN   
         Hillslope sub-surface soil                 NaN          NaN   
         Hillslope surface soil                     NaN          NaN   
         Link Initial Load                     0.000000     0.000000   
         Node Initial Load                     0.000000     0.000000   
         Point Source                               NaN    32.944679   
         Seepage                                    NaN          NaN   
         Streambank                          535.066403          NaN   
         Undefined     

In [13]:
#Calculate Totals - there must be a more elegant way of doing this!
#massBalanceAnnual_transposed['Total', 'Supply'] = massBalanceAnnual_transposed['Supply', 'Undefined'].fillna(0) + 
#                        massBalanceAnnual_transposed['Supply', 'Streambank'].fillna(0) + 
#                        massBalanceAnnual_transposed['Supply', 'Seepage'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Point Source'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Node Initial Load'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Link Initial Load'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Hillslope surface soil'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Hillslope sub-surface soil'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Hillslope no source distinction'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Gully'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Diffuse Dissolved'].fillna(0) +
#                        massBalanceAnnual_transposed['Supply', 'Channel Remobilisation'].fillna(0)
#massBalanceAnnual_transposed['Total', 'Loss'] = massBalanceAnnual_transposed['Loss', 'Extraction'].fillna(0)+ massBalanceAnnual_transposed['Loss', 'Flood Plain Deposition'].fillna(0) +  massBalanceAnnual_transposed['Loss', 'Node Loss'].fillna(0) + massBalanceAnnual_transposed['Loss', 'Reservoir Decay'].fillna(0) + massBalanceAnnual_transposed['Loss', 'Reservoir Deposition'].fillna(0) + massBalanceAnnual_transposed['Loss', 'Stream Decay'].fillna(0) + massBalanceAnnual_transposed['Loss', 'Stream Deposition'].fillna(0)
#massBalanceAnnual_transposed['Total', 'Residual'] = massBalanceAnnual_transposed['Residual', 'Residual Node Storage'].fillna(0) + massBalanceAnnual_transposed['Residual', 'Residual Link Storage'].fillna(0)
#massBalanceAnnual_transposed['Total', 'Other'] = massBalanceAnnual_transposed['Other', 'Leached'].fillna(0)
#massBalanceAnnual_transposed['Total', 'Export'] = massBalanceAnnual_transposed['Total', 'Supply'].fillna(0) - massBalanceAnnual_transposed['Total', 'Loss'].fillna(0) - massBalanceAnnual_transposed['Total', 'Residual'].fillna(0) - massBalanceAnnual_transposed['Total', 'Other'].fillna(0)
#massBalanceAnnual_transposed2 = massBalanceAnnual_transposed.transpose()
#massBalanceAnnual_transposed2

In [14]:
# Now.. as the last step, reorder the groups to a specified order...

final_mass_balance_table = massBalanceAnnual_transposed2.ix[['Supply','Loss','Other','Residual','Total']]
final_mass_balance_table

Constituent                               Sediment - Fine  Sediment - Coarse  \
Process  BudgetElement                                                         
Supply   Channel Remobilisation                  0.000000       0.000000e+00   
         Diffuse Dissolved                            NaN                NaN   
         Gully                                3693.380018       0.000000e+00   
         Hillslope no source distinction              NaN                NaN   
         Hillslope sub-surface soil              0.000000       0.000000e+00   
         Hillslope surface soil               1640.586204       1.370189e+08   
         Link Initial Load                       0.000000       0.000000e+00   
         Node Initial Load                       0.000000       0.000000e+00   
         Point Source                                 NaN                NaN   
         Seepage                                      NaN                NaN   
         Streambank                           1267.237651       8.062049e+08   
         Undefined                                    NaN                NaN   
Loss     Extraction                            563.051320       0.000000e+00   
         Flood Plain Deposition                  0.000000                NaN   
         Node Loss                            1331.856676       0.000000e+00   
         Reservoir Decay                              NaN                NaN   
         Reservoir Deposition                 1361.746308       0.000000e+00   
         Stream Decay                                 NaN                NaN   
         Stream Deposition                       0.000000       9.431344e+08   
Other    Leached                                      NaN                NaN   
Residual Residual Link Storage                  15.393046       5.368681e-04   
         Residual Node Storage                 296.663904       0.000000e+00   
Total    Supply                               6601.203874       9.432239e+08   
         Loss                                 3256.654304       9.431344e+08   
         Residual                              312.056950       5.368681e-04   
         Other                                   0.000000       0.000000e+00   
         Export                               3032.492620       8.950619e+04   

Constituent                               P_Particulate        P_FRP  \
Process  BudgetElement                                                 
Supply   Channel Remobilisation                0.000000          NaN   
         Diffuse Dissolved                          NaN  1420.531577   
         Gully                              1104.361263          NaN   
         Hillslope no source distinction    6098.053482          NaN   
         Hillslope sub-surface soil                 NaN          NaN   
         Hillslope surface soil                     NaN          NaN   
         Link Initial Load                     0.000000     0.000000   
         Node Initial Load                     0.000000     0.000000   
         Point Source                               NaN    32.944679   
         Seepage                                    NaN          NaN   
         Streambank                          535.066403          NaN   
         Undefined                           983.889665          NaN   
Loss     Extraction                         1199.083494    56.127939   
         Flood Plain Deposition                0.000000          NaN   
         Node Loss                          1923.651565   199.258786   
         Reservoir Decay                            NaN     0.000000   
         Reservoir Deposition               1954.967535          NaN   
         Stream Decay                               NaN     0.000000   
         Stream Deposition                     0.000000          NaN   
Other    Leached                                    NaN          NaN   
Residual Residual Link Storage                18.751432     4.434158   
         Residual Node 


_Now, most of this is captured in a standard query_

In [15]:
results.queries.annual_mass_balance()

Constituent                               Sediment - Fine  Sediment - Coarse  \
Process  BudgetElement                                                         
Supply   Channel Remobilisation                  0.000000       0.000000e+00   
         Diffuse Dissolved                            NaN                NaN   
         Gully                                3696.406159       0.000000e+00   
         Hillslope no source distinction              NaN                NaN   
         Hillslope sub-surface soil              0.000000       0.000000e+00   
         Hillslope surface soil               1641.930405       1.371312e+08   
         Link Initial Load                       0.000000       0.000000e+00   
         Node Initial Load                       0.000000       0.000000e+00   
         Point Source                                 NaN                NaN   
         Rainfall                                     NaN                NaN   
         Seepage                                      NaN                NaN   
         Streambank                           1268.275952       8.068655e+08   
         Undefined                                    NaN                NaN   
Loss     Evaporation                                  NaN                NaN   
         Extraction                            563.512651       0.000000e+00   
         Flood Plain Deposition                  0.000000                NaN   
         Infiltration                                 NaN                NaN   
         Node Loss                            1332.947922       0.000000e+00   
         Reservoir Decay                              NaN                NaN   
         Reservoir Deposition                 1362.862044       0.000000e+00   
         Stream Decay                                 NaN                NaN   
         Stream Deposition                       0.000000       9.439071e+08   
Other    Leached                                      NaN                NaN   
Residual Residual Link Storage                  15.405658       5.373079e-04   
         Residual Node Storage                 296.906973       0.000000e+00   
Total    Supply                               6606.612516       9.439967e+08   
         Loss                                 3259.322617       9.439071e+08   
         Residual                              312.312631       5.373079e-04   
         Other                                   0.000000       0.000000e+00   
         Export                               3034.977268       8.957953e+04   

Constituent                               P_Particulate        P_FRP  \
Process  BudgetElement                                                 
Supply   Channel Remobilisation                0.000000          NaN   
         Diffuse Dissolved                          NaN  1421.695478   
         Gully                              1105.266112          NaN   
         Hillslope no source distinction    6103.049873          NaN   
         Hillslope sub-surface soil                 NaN          NaN   
         Hillslope surface soil                     NaN          NaN   
         Link Initial Load                     0.000000     0.000000   
         Node Initial Load                     0.000000     0.000000   
         Point Source                               NaN    32.971672   
         Rainfall                                   NaN          NaN   
         Seepage                                    NaN          NaN   
         Streambank                          535.504806          NaN   
         Undefined                           984.695807          NaN   
Loss     Evaporation                                NaN          NaN   
         Extraction                         1200.065954    56.173927   
         Flood Plain Deposition                0.000000          NaN   
         Infiltration                               NaN          NaN   
         Node Loss                          1925.227693   199.422047   